# Ciencia de datos (CC408) - Primavera 2024

## Trabajo Práctico 1 - Parte 2

### Reglas de formato y presentation
- El trabajo debe tener una extensión máxima de 8 páginas (no se permite Apéndice). Se espera una buena redacción en la resolución del práctico.
- El informe debe ser entregado en formato PDF, con los gráficos e imágenes en este mismo archivo.
- Entregar el código con los comandos utilizados, identificando claramente a qué inciso corresponde cada comando.

### Fecha de entrega:
Miércoles 2 de octubre a las 23:59 hs.

### Modalidad de entrega
- Al finalizar el trabajo práctico deben hacer un último <i>commit</i> en su repositorio de GitHub llamado “Entrega final del tp”. 
- Asegurense de haber creado una carpeta llamada TP1. Este Jupyter Notebook y el correspondiente al TP1 - Parte 2 deben estar dentro de esa carpeta.
- También deben enviar el link de su repositorio -para que pueda ser clonado y corregido- al siguiente email: ispiousas@udesa.edu.ar
- La última versión en el repositorio es la que será evaluada. Por lo que es importante que: 
    - No envien el correo hasta no haber terminado y estar seguros de que han hecho el <i>commit y push</i> a la versión final que quieren entregar. Debido a que se pueden tomar hasta 3 días de extensión a lo largo del curso, no se corregirán sus tareas hasta no recibir el correo.
    - No hagan nuevos <i>push</i> despues de haber entregado su versión final. Esto generaría confusión acerca de que versión es la que quieren que se les corrija. 

### Parte I: Limpieza de la base

#### Ejercicio 1
La base de datos provista contiene información sobre oferentes de Airbnb en la ciudad de Nueva York. Realicen una limpieza de la base.
- Tengan en cuenta si hay valores duplicados y eliminénlos.
- Eliminen las columnas que no tienen información de interés.
- (Opcional) Luego de leer el artículo __[Missing-data imputation](https://bookdown.org/mike/data_analysis/imputation-missing-data.html##imputation-missing-data)__ (__[Missing-data imputation articulo alternativo](http://www.stat.columbia.edu/~gelman/arm/missing.pdf)__), decidan qué hacer con los missing values e implementen su decision. Justifiquen su eleccion. Pueden usar informacion del paper mencionado o de otras fuentes (citenlas si las usan) para explicar los problemas que pueden surgir por su estrategia elegida.
- Si hay observaciones con *outliers* o valores que no tienen sentido, tomen una decision. Expliquen las decisiones tomadas.
- Transformen las variables **‘neighbourhood_group’** y **‘room_type’** a variables numericas.
- Con la ayuda de los comandos <code>groupby</code> y <code>join</code> o <code>merge</code>, creen una columna que tenga la cantidad de oferentes por “Neighbourhood group”. Llamen a esa nueva columna <code>offer group</code>

In [ ]:
# Resolver acá

import pandas as pd 

df_airbnb = pd.read_csv("C:/Users/marcos.olavarria/Documents/GitHub/CC408/TP 2/Base Airbnb NY.csv")
df_airbnb


# Identificar filas duplicadas
duplicados = df_airbnb.drop_duplicates(subset="id")
duplicados


# 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



columnas_a_transformar = ['price']
for col in columnas_a_transformar:
    df_airbnb[f'log_{"price"}'] = np.log(df_airbnb["price"])  

plt.figure(figsize=(10, 6))
# Histograma
sns.histplot(df_airbnb['price'], bins=10, kde=True)  # kde=True para agregar la densidad
plt.title('Distribución del Logaritmo del Precio')
plt.xlabel('Price')
plt.ylabel('Frecuencia')
plt.grid()

plt.show()

### Parte II: Graficos y visualizaciones

#### Ejercicio 2
Una vez hecha la limpieza, realicen una matriz de correlación con las siguientes variables:**‘neighbourhood group’, ‘latitude’, ‘longitude’, ‘room type’,‘price’, ‘minimum nights’, ‘number of reviews’, ‘reviews per month’,‘calculated host listings count’, ‘availability 365’.** Comenten los resultados. Utilicen alguno de los comandos disponibles en este link para graficar la matriz de correlacion.

*Nota:* consideren cómo es conveniente incluir las variables que originalmente eran categóricas para poder interpretar mejor la matriz de correlación.

In [ ]:
# Resolver acá


#### Ejercicio 3
Respondan las siguientes preguntas: ¿Cuál es la proporción de oferentes por “Neighbourhood group”? ¿Y por tipo de habitación? Además, realicen gráficos para mostrar estas composiciones y comenten los resultados.

In [ ]:
# Resolver acá
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

neighbourhood_counts = df_airbnb['neighbourhood_group'].value_counts(normalize=True)
print(neighbourhood_counts)

room_type_counts = df_airbnb['room_type'].value_counts(normalize=True)
print(room_type_counts)

plt.figure(figsize=(10, 6))
sns.barplot(x=neighbourhood_counts.index, y=neighbourhood_counts.values)
plt.title('Proporción de Oferentes por Neighbourhood Group')
plt.xlabel('Neighbourhood Group')
plt.ylabel('Proporción')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x=room_type_counts.index, y=room_type_counts.values)
plt.title('Proporción de Oferentes por Tipo de Habitación')
plt.xlabel('Tipo de Habitación')
plt.ylabel('Proporción')
plt.xticks(rotation=45)
plt.show()


#### Ejercicio 4
Realicen un histograma de los precios de los alojamientos. Comenten el gráfico obtenido. Además, respondan las siguientes preguntas: ¿cuál es el precio mínimo, máximo y promedio? ¿Cuál es la media de precio por “Neighbourhood group” y por tipo de habitación? 

In [ ]:
# Resolver acá


#### Ejercicio 5
Realicen dos scatter plots con dos variables de interés en cada uno. Comenten.

In [1]:
# Resolver acá


#### Ejercicio 6
Utilicen el análisis de componentes principales para graficar las variables en dos dimensiones. Comenten los resultados obtenidos (qué porcentaje de la varianza se logra explicar con dos componentes, cómo son los *loadings*, si ven algún patrón en el gráfico).

In [1]:
# Resolver acá


### Parte III: Predicción

El objetivo de esta parte del trabajo es intentar predecir los precios de los alojamientos.

#### Ejercicio 7
Eliminen de la base todas las variables relacionadas al precio

In [ ]:
# Resolver acá
#NO HAGAMOS EL 7

columns_to_remove = ['price', 'minimum_nights']

# Eliminar las columnas del DataFrame
df_airbnb_cleaned = df_airbnb.drop(columns=columns_to_remove)

# Mostrar las primeras filas del DataFrame limpio
print(df_airbnb_cleaned.head())


#### Ejercicio 8
Partan la base en una base de prueba (test) y una de entrenamiento (train) utilizando el comando <code>train_test_split</code>. La base de entrenamiento debe comprender el 70% de los datos, y la semilla a utilizar (*random state instance*) debe ser 201. Establezca a price como su variable dependiente en la base de entrenamiento (vector y). El resto de las variables ser´an las variables independientes (matriz X). Recuerden agregar la columna de unos (1).

In [42]:
# Resolver acá
from sklearn.model_selection import train_test_split
import pandas as pd

# Definir la variable dependiente (y) y las independientes (X)
y = df_airbnb['price']  # Asegúrate de que 'price' sea la variable correcta
X = df_airbnb.drop(columns=['id','name','host_id','host_name','neighbourhood_group','neighbourhood','latitude','longitude','room_type','minimum_nights','number_of_reviews','last_review','reviews_per_month','calculated_host_listings_count','availability_365'])  # Eliminar la variable dependiente de X

# Partir la base de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=201)

# Agregar una columna de unos (intercepto) a la matriz X
import numpy as np

X_train = np.c_[np.ones(X_train.shape[0]), X_train]  # Añade columna de unos al conjunto de entrenamiento
X_test = np.c_[np.ones(X_test.shape[0]), X_test]      # Añade columna de unos al conjunto de prueba

# Mostrar las dimensiones de los conjuntos
print("Dimensiones del conjunto de entrenamiento X:", X_train.shape)
print("Dimensiones del conjunto de prueba X:", X_test.shape)
print("Dimensiones del conjunto de entrenamiento y:", y_train.shape)
print("Dimensiones del conjunto de prueba y:", y_test.shape)


Dimensiones del conjunto de entrenamiento X: (34233, 3)
Dimensiones del conjunto de prueba X: (14672, 3)
Dimensiones del conjunto de entrenamiento y: (34233,)
Dimensiones del conjunto de prueba y: (14672,)


#### Ejercicio 9
Implementen una regresión lineal y comenten los resultados obtenidos.

In [ ]:
# Resolver acá
from sklearn.linear_model import LinearRegression


# Crear una instancia del modelo de regresión lineal
model = LinearRegression()

# Ajustar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Hacer predicciones sobre el conjunto de prueba
y_pred = model.predict(X_test)


from sklearn.metrics import mean_squared_error, r2_score

# Calcular MSE y R²
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error Cuadrático Medio (MSE): {mse}")
print(f"Coeficiente de Determinación (R²): {r2}")


# Coeficientes del modelo
coefficients = model.coef_
feature_names = X_train.columns  # Ajusta si es necesario

# Crear un DataFrame para mostrar los coeficientes
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
print(coef_df)

